In [15]:
%pip install spacy

   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ------------ --------------------------- 3.9/12.2 MB 19.5 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.2 MB 22.6 MB/s eta 0:00:01
   ------------------------------------ --- 11.3/12.2 MB 18.0 MB/s eta 0:00:01
   ---------------------------------------- 12.2/12.2 MB 17.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 21.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 19.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ------------ --------------------------- 5.0/15.9 MB 27.4 MB/s eta 0:00:01
   ---------------------------- ----------- 11.5/15.9 MB 27.7 MB/s eta 0:00:01
   ---------------------------------------- 15.9/15.9 MB 26.3 MB/s eta 0:00:00
   ---------------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.20.0 requires xarray>=2022.6.0, which is not installed.
lightweight-mmm 0.1.9 requires statsmodels>=0.13.0, which is not installed.
matplotlib 3.6.1 requires contourpy>=1.0.1, which is not installed.
matplotlib 3.6.1 requires cycler>=0.10, which is not installed.
matplotlib 3.6.1 requires fonttools>=4.22.0, which is not installed.
matplotlib 3.6.1 requires kiwisolver>=1.0.1, which is not installed.
matplotlib 3.6.1 requires pillow>=6.2.0, which is not installed.
matplotlib 3.6.1 requires pyparsing>=2.2.1, which is not installed.
numpyro 0.15.3 requires multipledispatch, which is not installed.
xarray-einstats 0.8.0 requires xarray>=2022.09.0, which is not installed.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 w

In [23]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-uncased-contracts")

# Load the model
model = AutoModelForMaskedLM.from_pretrained("nlpaueb/bert-base-uncased-contracts")

text ="The [MASK] shall indemnify and hold harmless the other party from any claims arising from the contract."

# Tokenize input
inputs = tokenizer(text, return_tensors="pt")
# Get model predictions
with torch.no_grad():
    outputs = model(**inputs)

# Extract logits
logits = outputs.logits
# Find the index of [MASK] token
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
# Get logits for the [MASK] token
mask_token_logits = logits[0, mask_token_index, :]

# Get the top 5 predictions
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()


# Decode the top 5 predictions
for token in top_5_tokens:
    predicted_token = tokenizer.decode([token])
    print(f"Predicted token: {predicted_token}")





Predicted token: parties
Predicted token: party
Predicted token: company
Predicted token: contractor
Predicted token: recipient


In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-uncased-contracts")

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/bert-base-uncased-contracts", num_labels=2)
text = "This agreement is made between the seller and the buyer for the sale of goods."

# Tokenize input
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
# Get model predictions
with torch.no_grad():
    outputs = model(**inputs)

# Extract logits
logits = outputs.logits
from torch.nn.functional import softmax

# Apply softmax to get probabilities
probabilities = softmax(logits, dim=1)

# Get the predicted class
predicted_class = torch.argmax(probabilities, dim=1).item()

# Define class labels
class_labels = ["Non-Contract", "Contract"]

# Print the result
print(f"Predicted class: {class_labels[predicted_class]}")




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/bert-base-uncased-contracts and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: Non-Contract
